In [86]:
import urllib.request
import tarfile
import urllib
from Bio import Entrez
from xml.dom import minidom
import xml.etree.ElementTree as ET
import pandas as pd
import os
import glob
import shutil
import urllib.request as request
from contextlib import closing
import pickle

In [87]:
# 1. goal: summarization for articles that do not have abstract
# 2. mesh: sub-categories
# 3. pubmed pmc[sb] - restrict to articles that have full text in PMC
# 4. cluster: title only / fulltext only / abstract & title only ...  - sperate them 
# 5. ignore the case as https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1248604/pdf/bmjcred00049-0017.pdf 
# 6. two dataset: a. for publication types (following multitagger paper) 1987 - 2023 b. for sumarization (depends on pubmed central)
# 7. standard dataset: 
# 8. remind halil to create the github repo


In [88]:
import os

In [125]:
year = "2023"

In [111]:
current_directory = os.getcwd() + "/data_summarization_with_abstract/" + year

In [112]:
test_xml = os.listdir(current_directory)[0]

In [113]:
tree = ET.parse(os.path.join(os.getcwd() + "/data_summarization_with_abstract/" + year, test_xml))
root = tree.getroot()

In [114]:
# the year / date  <PubDate>
# journal title / issn
# page information (start / end) - to calculate how long the article is
# the <meshheading> list
# publication type list <PublicationType>
# doi - for special cases 

test_count = 1000000
count = 0
pmids = []
pmcids = []
Article_Title = []
Article_Abstract = []

PubDate = []
Journal_ISSN = []
Journal_Title = []
StartPage = []
EndPage = []
MeshHeading = []
PublicationType = []
doi = []
KeywordList = []


for child in root:
    if count < test_count:
        if child.tag == "PubmedArticle":
            pmid = ""
            title = ""
            pmcid = ""
            abstract = ""
            pub_date = ""
            issn = ""
            journal_title = ""
            start_page = ""
            end_page = ""
            mesh_heading = []
            publication_type = []
            single_doi = ""
            keyword_list = []
            
            count = count + 1
            for child_level_2 in child:
                if "MedlineCitation" in child_level_2.tag:
                    for child_level_3 in child_level_2:
                        if ("PMID" in child_level_3.tag):
                            pmid = child_level_3.text
                                    
                        if "MeshHeadingList" in child_level_3.tag:
                            for child_level_4 in child_level_3:
                                if "MeshHeading" in child_level_4.tag:
                                    for child_level_5 in child_level_4:
                                        if "DescriptorName" in child_level_5.tag:
                                            mesh_heading.append(child_level_5.text)
                                            
                        if "KeywordList" in child_level_3.tag:
                            for child_level_4 in child_level_3:
                                if "Keyword" in child_level_4.tag:
                                    keyword_list.append(child_level_4.text)
                                            
                        if ("Article" in child_level_3.tag):
                            for child_level_4 in child_level_3:
                                if "ArticleTitle" in child_level_4.tag:
                                    title = child_level_4.text
                                if "Abstract" in child_level_4.tag:
                                    for child_level_5 in child_level_4:
                                        if "AbstractText" in child_level_5.tag:
                                            abstract = child_level_5.text
                                if ("Journal" in child_level_4.tag):
                                    for child_level_5 in child_level_4:
                                        if "ISSN" in child_level_5.tag: 
                                            issn = child_level_5.text
                                        if "Title" in child_level_5.tag:
                                            journal_title = child_level_5.text
                                        if "JournalIssue" in child_level_5.tag:
                                            for child_level_6 in child_level_5:
                                                if "PubDate" in child_level_6.tag:
                                                    for child_level_7 in child_level_6:
                                                        if "Year" in child_level_7.tag:
                                                            pub_date = pub_date + child_level_7.text
                                                        if "Month" in child_level_7.tag:
                                                            pub_date = pub_date + ", " + child_level_7.text
                                if "Pagination" in child_level_4.tag:
                                    for child_level_5 in child_level_4:
                                        if "StartPage" in child_level_5.tag:
                                            start_page = child_level_5.text
                                        if "EndPage" in child_level_5.tag:
                                            end_page = child_level_5.text
                                if "PublicationTypeList" in child_level_4.tag:
                                    for child_level_5 in child_level_4:
                                        if "PublicationType" in child_level_5.tag:
                                            publication_type.append(child_level_5.text)
                                            
                elif "PubmedData" in child_level_2.tag:
                    for child_level_3 in child_level_2:
                        if ("ArticleIdList" in child_level_3.tag):
                            for child_level_4 in child_level_3:
                                if child_level_4.attrib["IdType"] == "pmc":
                                    pmcid = child_level_4.text
                                if child_level_4.attrib["IdType"] == "doi":
                                    single_doi = child_level_4.text
            pmids.append(pmid)
            Article_Title.append(title)
            pmcids.append(pmcid)
            Article_Abstract.append(abstract)
            
            PubDate.append(pub_date)
            Journal_ISSN.append(issn)
            Journal_Title.append(journal_title)
            StartPage.append(start_page)
            EndPage.append(end_page)
            MeshHeading.append(mesh_heading)
            PublicationType.append(publication_type)
            doi.append(single_doi)
            KeywordList.append(keyword_list)
            

In [126]:
df.to_csv(year + "_metadata.csv")

In [115]:
df = pd.DataFrame(list(zip(pmids, pmcids, PubDate, Journal_ISSN, Journal_Title, EndPage, MeshHeading, PublicationType, StartPage, Article_Title, Article_Abstract, doi, KeywordList)), columns =["pmids", "pmcids", "PubDate", "Journal_ISSN", "Journal_Title", "EndPage", "MeshHeading", "PublicationType", "StartPage", "Article_Title", "Article_Abstract", "doi", "KeywordList"])

In [117]:
df

,pmids,pmcids,PubDate,Journal_ISSN,Journal_Title,EndPage,MeshHeading,PublicationType,StartPage,Article_Title,Article_Abstract,doi,KeywordList
0,37811508,PMC10551689,"2023, Oct",2005-4408,Clinics in orthopedic surgery,733,"[Female, Humans, Adolescent, Finite Element An...","[Case Reports, Journal Article]",725,Femoral Bowing Increases Early Postoperative S...,Surgeons should assess femoral bowing preopera...,10.4055/cios22392,"[Bowing, Femur, Finite element analysis, Stem,..."
1,37808910,PMC10552297,2023,2235-2988,Frontiers in cellular and infection microbiology,,"[Female, Humans, Aged, Aged, 80 and over, Myco...","[Case Reports, Research Support, Non-U.S. Gov't]",1268668,Cure of,We report a case of,10.3389/fcimb.2023.1268668,"[Mycobacterium morphology, keratitis, laborato..."
2,37808435,PMC10559155,2023,1937-8688,The Pan African medical journal,,"[Male, Humans, Adult, Middle Aged, Aged, Aged,...",[Case Reports],144,Contrast media-induced nephropathy in Tunisia:...,the most effective approach for CMIN is preven...,10.11604/pamj.2023.45.144.30749,"[Acute renal failure, iodinated contrast media..."
3,37805489,PMC10560414,"2023, Oct",1752-1947,Journal of medical case reports,,"[Humans, Female, Aged, Ureter, Ureteral Diseas...","[Case Reports, Journal Article]",443,Localized amyloidosis of the ureter: a case re...,We herein report a rare case of localized uret...,10.1186/s13256-023-04138-y,"[Amyloidosis, Localized amyloidosis, Ureteral ..."
4,37805478,PMC10560420,"2023, Oct",1749-8090,Journal of cardiothoracic surgery,,"[Humans, Female, Aged, 80 and over, Percutaneo...","[Case Reports, Journal Article]",275,Successful conservative treatment for left ven...,Left ventricular free wall rupture (LVFWR) is ...,10.1186/s13019-023-02397-w,"[Acute myocardial infarction (AMI), Cardiac ta..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6003,33713550,PMC10029896,"2023, Mar",2053-1095,Veterinary medicine and science,716,"[Humans, Animals, Horses, Aortic Valve, Quadri...",[Case Reports],712,Transthoracic echocardiography and its limitat...,Aortic valve malformation is a common congenit...,10.1002/vms3.472,"[aortic malformation, congenital abnormality, ..."
6004,33709828,PMC10076152,"2023, Feb",1724-6075,Urologia,197,"[Humans, Male, Middle Aged, Adjuvants, Immunol...","[Review, Case Reports, Journal Article]",195,Disease management in a patient diagnosed with...,This group of patients' recommendation is to d...,10.1177/03915603211001670,"[Bladder cancer, COVID-19, coronavirus, intrav..."
6005,33394953,PMC9750092,"2023, Jan",1937-1578,Retinal cases & brief reports,22,"[Male, Humans, Aged, Melanoma, Skin Neoplasms,...","[Case Reports, Journal Article]",18,VOGT-KOYANAGI-HARADA-LIKE UVEITIS FOLLOWED BY ...,Patients with metastatic cutaneous melanoma ca...,10.1097/ICB.0000000000001113,[]
6006,33323898,PMC9750093,"2023, Jan",1937-1578,Retinal cases & brief reports,43,"[Female, Humans, Middle Aged, Scleral Buckling...","[Case Reports, Journal Article]",41,INTRACTABLE PAIN AND DIPLOPIA AFTER SCLERAL BU...,Spontaneous dehiscence of scleral belt loops m...,10.1097/ICB.0000000000001104,[]


In [118]:
df_filtered = df[df.pmcids != ""]



In [119]:
df_filtered



,pmids,pmcids,PubDate,Journal_ISSN,Journal_Title,EndPage,MeshHeading,PublicationType,StartPage,Article_Title,Article_Abstract,doi,KeywordList
0,37811508,PMC10551689,"2023, Oct",2005-4408,Clinics in orthopedic surgery,733,"[Female, Humans, Adolescent, Finite Element An...","[Case Reports, Journal Article]",725,Femoral Bowing Increases Early Postoperative S...,Surgeons should assess femoral bowing preopera...,10.4055/cios22392,"[Bowing, Femur, Finite element analysis, Stem,..."
1,37808910,PMC10552297,2023,2235-2988,Frontiers in cellular and infection microbiology,,"[Female, Humans, Aged, Aged, 80 and over, Myco...","[Case Reports, Research Support, Non-U.S. Gov't]",1268668,Cure of,We report a case of,10.3389/fcimb.2023.1268668,"[Mycobacterium morphology, keratitis, laborato..."
2,37808435,PMC10559155,2023,1937-8688,The Pan African medical journal,,"[Male, Humans, Adult, Middle Aged, Aged, Aged,...",[Case Reports],144,Contrast media-induced nephropathy in Tunisia:...,the most effective approach for CMIN is preven...,10.11604/pamj.2023.45.144.30749,"[Acute renal failure, iodinated contrast media..."
3,37805489,PMC10560414,"2023, Oct",1752-1947,Journal of medical case reports,,"[Humans, Female, Aged, Ureter, Ureteral Diseas...","[Case Reports, Journal Article]",443,Localized amyloidosis of the ureter: a case re...,We herein report a rare case of localized uret...,10.1186/s13256-023-04138-y,"[Amyloidosis, Localized amyloidosis, Ureteral ..."
4,37805478,PMC10560420,"2023, Oct",1749-8090,Journal of cardiothoracic surgery,,"[Humans, Female, Aged, 80 and over, Percutaneo...","[Case Reports, Journal Article]",275,Successful conservative treatment for left ven...,Left ventricular free wall rupture (LVFWR) is ...,10.1186/s13019-023-02397-w,"[Acute myocardial infarction (AMI), Cardiac ta..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6003,33713550,PMC10029896,"2023, Mar",2053-1095,Veterinary medicine and science,716,"[Humans, Animals, Horses, Aortic Valve, Quadri...",[Case Reports],712,Transthoracic echocardiography and its limitat...,Aortic valve malformation is a common congenit...,10.1002/vms3.472,"[aortic malformation, congenital abnormality, ..."
6004,33709828,PMC10076152,"2023, Feb",1724-6075,Urologia,197,"[Humans, Male, Middle Aged, Adjuvants, Immunol...","[Review, Case Reports, Journal Article]",195,Disease management in a patient diagnosed with...,This group of patients' recommendation is to d...,10.1177/03915603211001670,"[Bladder cancer, COVID-19, coronavirus, intrav..."
6005,33394953,PMC9750092,"2023, Jan",1937-1578,Retinal cases & brief reports,22,"[Male, Humans, Aged, Melanoma, Skin Neoplasms,...","[Case Reports, Journal Article]",18,VOGT-KOYANAGI-HARADA-LIKE UVEITIS FOLLOWED BY ...,Patients with metastatic cutaneous melanoma ca...,10.1097/ICB.0000000000001113,[]
6006,33323898,PMC9750093,"2023, Jan",1937-1578,Retinal cases & brief reports,43,"[Female, Humans, Middle Aged, Scleral Buckling...","[Case Reports, Journal Article]",41,INTRACTABLE PAIN AND DIPLOPIA AFTER SCLERAL BU...,Spontaneous dehiscence of scleral belt loops m...,10.1097/ICB.0000000000001104,[]


In [120]:
pmcids = df_filtered.pmcids.to_list()

In [121]:
pmcids

['PMC10551689',
 'PMC10552297',
 'PMC10559155',
 'PMC10560414',
 'PMC10560420',
 'PMC10560406',
 'PMC10557268',
 'PMC10559408',
 'PMC10559591',
 'PMC10559464',
 'PMC10557309',
 'PMC10558226',
 'PMC10553092',
 'PMC10553104',
 'PMC10552958',
 'PMC10553151',
 'PMC10548267',
 'PMC10557153',
 'PMC10557175',
 'PMC10557371',
 'PMC10557266',
 'PMC10552249',
 'PMC9820790',
 'PMC10547563',
 'PMC10552215',
 'PMC10560792',
 'PMC10542919',
 'PMC10543984',
 'PMC10543903',
 'PMC10543906',
 'PMC10544010',
 'PMC10548661',
 'PMC10546625',
 'PMC10547717',
 'PMC10551871',
 'PMC10527838',
 'PMC10549188',
 'PMC10546088',
 'PMC10541542',
 'PMC10560793',
 'PMC10546704',
 'PMC10544472',
 'PMC10544493',
 'PMC10546703',
 'PMC10546652',
 'PMC10546740',
 'PMC10556538',
 'PMC10476227',
 'PMC10535097',
 'PMC10537924',
 'PMC10536161',
 'PMC10533992',
 'PMC10539909',
 'PMC10524816',
 'PMC10524814',
 'PMC10538626',
 'PMC10539003',
 'PMC10539002',
 'PMC10539004',
 'PMC10539001',
 'PMC10539005',
 'PMC10556539',
 'PMC1054

In [122]:
tarfiles_list = []
have_link = []
for PMCID in pmcids:
    print (PMCID)
    if PMCID != "":
        # call PMC API here
        link = "https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=" + PMCID
        print (link)
        #35640925
        # open the link
        request = urllib.request.Request(link)
        result = urllib.request.urlopen(request)
        tree = ET.parse(result)
        doc = tree.getroot()
        for child in doc:
            if child.tag == "records":
                article_download_link = ""
                for child_level_2 in child:
                    if child_level_2.tag == "record":
                        for child_level_3 in child_level_2:
                            if ("link" in child_level_3.tag):
                                link_format = child_level_3.attrib["format"]
                                if (article_download_link=="") and (link_format == "tgz"):
                                    article_download_link = child_level_3.attrib["href"]
                                    
                # print out the article tar folder download link
                print (article_download_link)
                
                ftpstream = urllib.request.urlopen(article_download_link)
                thetarfile = tarfile.open(fileobj=ftpstream, mode="r|gz")
                
                # put the path of output folder here
                thetarfile.extractall(path= current_directory + "/NXML_second/")

PMC10551689
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10551689
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/b2/8b/PMC10551689.tar.gz
PMC10552297
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10552297
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/20/65/PMC10552297.tar.gz
PMC10559155
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10559155
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/6e/86/PMC10559155.tar.gz
PMC10560414
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10560414
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/c7/4a/PMC10560414.tar.gz
PMC10560420
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10560420
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/54/3d/PMC10560420.tar.gz
PMC10560406
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10560406
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/c5/4f/PMC10560406.tar.gz
PMC10557268
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10557268
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/

PMC10539003
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10539003
PMC10539002
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10539002
PMC10539004
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10539004
PMC10539001
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10539001
PMC10539005
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10539005
PMC10556539
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10556539
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/02/c7/PMC10556539.tar.gz
PMC10544463
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10544463
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/a8/14/PMC10544463.tar.gz
PMC10543861
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10543861
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/ba/32/PMC10543861.tar.gz
PMC10543300
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10543300
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/fc/5b/PMC10543300.tar.gz
PMC10543269
https://www

PMC10534796
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10534796
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/a6/40/PMC10534796.tar.gz
PMC10532981
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10532981
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/7b/7b/PMC10532981.tar.gz
PMC10533146
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10533146
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/2f/22/PMC10533146.tar.gz
PMC10532794
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10532794
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/b3/52/PMC10532794.tar.gz
PMC10533137
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10533137
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/59/9b/PMC10533137.tar.gz
PMC10531046
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10531046
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/49/e0/PMC10531046.tar.gz
PMC10530954
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10530954
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/

PMC10512180
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10512180
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/7c/0d/PMC10512180.tar.gz
PMC10534165
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10534165
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/75/25/PMC10534165.tar.gz
PMC10519003
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10519003
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/ad/f2/PMC10519003.tar.gz
PMC10534166
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10534166
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/0b/c7/PMC10534166.tar.gz
PMC10518083
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10518083
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/62/53/PMC10518083.tar.gz
PMC10518089
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10518089
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/91/17/PMC10518089.tar.gz
PMC10517471
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10517471
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/

PMC10507825
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10507825
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/44/6f/PMC10507825.tar.gz
PMC10508002
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10508002
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/a1/13/PMC10508002.tar.gz
PMC10510232
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10510232
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/cd/7f/PMC10510232.tar.gz
PMC10510258
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10510258
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/c9/00/PMC10510258.tar.gz
PMC10507818
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10507818
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/d0/a4/PMC10507818.tar.gz
PMC10507849
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10507849
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/63/e5/PMC10507849.tar.gz
PMC10505428
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10505428
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/

...

PMC10076128
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10076128
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/b2/5e/PMC10076128.tar.gz
PMC10076138
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10076138
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/54/9f/PMC10076138.tar.gz
PMC9892383
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC9892383
PMC9388350
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC9388350
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/6b/e8/PMC9388350.tar.gz
PMC9987742
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC9987742
PMC10087263
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10087263
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/bb/fb/PMC10087263.tar.gz
PMC9389272
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC9389272
PMC9389274
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC9389274
PMC9902188
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC9902188
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/o

PMC10037001
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10037001
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/b4/77/PMC10037001.tar.gz
PMC9308502
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC9308502
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/5a/80/PMC9308502.tar.gz
PMC9892406
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC9892406
PMC9307428
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC9307428
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/f3/7a/PMC9307428.tar.gz
PMC9912182
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC9912182
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/de/1f/PMC9912182.tar.gz
PMC9833369
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC9833369
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/e5/35/PMC9833369.tar.gz
PMC9350234
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC9350234
PMC9296111
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC9296111
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/6

PMC9908397
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC9908397
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/a3/d5/PMC9908397.tar.gz
PMC9908399
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC9908399
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/22/25/PMC9908399.tar.gz
PMC9876703
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC9876703
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/20/78/PMC9876703.tar.gz
PMC9876725
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC9876725
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/53/c3/PMC9876725.tar.gz
PMC9908398
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC9908398
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/32/2c/PMC9908398.tar.gz
PMC10017242
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10017242
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/be/c2/PMC10017242.tar.gz
PMC9809993
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC9809993
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/57/36/

ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/9e/4c/PMC9873340.tar.gz
PMC9985565
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC9985565
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/e3/0c/PMC9985565.tar.gz
PMC9812411
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC9812411
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/82/74/PMC9812411.tar.gz
PMC9938053
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC9938053
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/d0/20/PMC9938053.tar.gz
PMC10362440
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10362440
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/eb/10/PMC10362440.tar.gz
PMC10466116
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10466116
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/cc/76/PMC10466116.tar.gz
PMC10065769
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10065769
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/6b/21/PMC10065769.tar.gz
PMC8903471
https://www.ncbi.nlm.nih.gov/pmc/utils/o

PMC10362432
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10362432
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/26/a3/PMC10362432.tar.gz
PMC10362431
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10362431
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/ac/c9/PMC10362431.tar.gz
PMC9801191
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC9801191
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/39/a3/PMC9801191.tar.gz
PMC10251175
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10251175
PMC10251170
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10251170
PMC10121363
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10121363
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/10/5c/PMC10121363.tar.gz
PMC9949519
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC9949519
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/a2/eb/PMC9949519.tar.gz
PMC10388251
https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC10388251
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/